# Part 2: Time Series Modeling

In this notebook, you will implement functions to extract features from time series data and build ARIMA models.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from pathlib import Path
import os

## 1. Feature Extraction

Implement the `extract_time_series_features` function to calculate rolling window features.

In [ ]:
import pandas as pd

def extract_time_series_features(data, window_size=60):
    """Extract rolling window features from time series data.
    
    Parameters
    ----------
    data : pd.DataFrame
        Preprocessed physiological data
    window_size : int
        Size of the rolling window in seconds
        
    Returns
    -------
    pd.DataFrame
        DataFrame containing extracted features for each signal
    """
    data = data.set_index('timestamp')
    data = data.sort_index()

    ts_name = ['heart_rate', 'eda', 'temperature']
    rolling = pd.DataFrame()
    
    for ts in ts_name:
        rolling0 = data[ts].rolling(window=window_size)
        rolling[f'{ts}_mean'] = rolling0.mean()
        rolling[f'{ts}_std'] = rolling0.std()
        rolling[f'{ts}_min'] = rolling0.min()
        rolling[f'{ts}_max'] = rolling0.max()
        
        # Calculate autocorrelation
        def autocorr(x):
            return x.autocorr(lag=1)
        rolling[f'{ts}_autocorr1'] = rolling0.apply(autocorr)
    
    feature_df = rolling.dropna()
    feature_df = feature_df.reset_index()
    return feature_df

## 2. ARIMA Modeling

Implement the `build_arima_model` function to fit ARIMA models and generate diagnostic plots.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA  # ARIMA from statsmodels.tsa.arima.model
from pathlib import Path
import os

def build_arima_model(series, order=(1,1,1), output_dir='plots'):
    """Fit an ARIMA model to the time series and generate diagnostic plots.
    
    Parameters
    ----------
    series : pd.Series
        Time series data to model
    order : tuple
        (p,d,q) order of the ARIMA model
    output_dir : str
        Directory to save diagnostic plots
        
    Returns
    -------
    statsmodels.tsa.arima.model.ARIMAResults
        Fitted ARIMA model
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Fit ARIMA model
    model = ARIMA(series, order=order)
    model_fit = model.fit()
    
    print(model_fit.summary())
    
    # Generate diagnostic plots:
    # Model fit plot
    plt.figure(figsize=(12, 8))
    plt.subplot(211)
    plt.plot(series, label='Observed')
    plt.plot(model_fit.fittedvalues, color='red', label='Fitted')
    plt.title('ARIMA Model Fit')
    plt.legend()
    plt.savefig(os.path.join(output_dir, 'arima_model_fit.png'))
    plt.close()
    
    # Residuals plot
    plt.figure(figsize=(12, 8))
    plt.subplot(212)
    plt.plot(model_fit.resid, label='Residuals')
    plt.title('Residuals of ARIMA Model')
    plt.legend()
    plt.savefig(os.path.join(output_dir, 'arima_residuals.png'))
    plt.close()
    
    # Forecast plot
    forecast = model_fit.get_forecast(steps=30)
    forecast_index = pd.date_range(start=series.index[-1], periods=30, freq='S')
    forecast_series = pd.Series(forecast.predicted_mean, index=forecast_index)
    plt.figure(figsize=(12, 8))
    plt.plot(series, label='Observed')
    plt.plot(forecast_series, color='red', label='Forecast')
    plt.fill_between(forecast_index, forecast.conf_int().iloc[:, 0], forecast.conf_int().iloc[:, 1], color='pink', alpha=0.5)
    plt.title('ARIMA Forecast')
    plt.legend()
    plt.savefig(os.path.join(output_dir, 'arima_forecast.png'))
    plt.close()

    return model
